# Вычисление



# Устройство Машины Тьюринга

Рассмотрим простейшив вид машин Тьюринга. Машина Тьюринга состоит из бесконечной **ленты** (*tape*), разбитой на **ячейки** (*cells*), **головки** (*head*), которая указывает на ячейку ленты, и способной считывать или записывать символ, и **регистра состояний** (*state register*), в котором хранится текущее состояние машины. Число возможных состояний $q_1, \ldots, q_m$ машины конечно. Среди них имеется **стартовое состояние** (*start state*), которая инициализирует регистр состояний.

Набор возможных **действий** (*actions*):
1. Сдвиг головки вправо на один шаг ($R$);
2. Сдвиг головки влево на один шаг ($L$);
3. Остановка вычислений (halt);
4. Стереть запись ячейки (записать символ $S_0$);
5. Записать символ $S_1$.

В общем случае, если доступно больше символов $S_2, S_3, \ldots, S_n$, то список действий нужно дополнять. Так, список действий будет состоять из $4+n$ пунктов. Далее будем предполагать, что $n=1$, т.е. в наличии имеются два символа: пустая ячейка $S_0$ и заштрихованная ячейка $S_1$.

Каждое **состояние** $q_i$ представляет собой условную инструкцию: в зависимости от считываемого на текущем этапе символа, инструкция определяет какое действие будет произведено и в какое состояние перейдет машина. Т.о. Поведение машины на произвольном этапе определяется значением в регистре состояния и значением ячейки, на которую указывает головка.

Конкретная машина Тьюринга определяется не только устройством, но и программой. Каждая программа определяет конкретную машину Тьюринга (рассматриваемого вида).


# Программа

Набор состояний $q_1, \ldots, q_m$ определяет программу. В качестве стартового состояния принято использовать $q_1$. Есть несколько способов описания программы: **машинная таблица** (*machine table*), **граф переходов** (*flow chart* или *flow graph*) и **множество четверок** (*set of quadruples*). 

Рассмотрим для примера простую программу, которая записывает три символа $S_1$ на пустой ленте и завершается, сканируя крайний левый из этих трех символов. Три способа представления такой программы:

<img src="https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/science/turing_machine_program.png">

Машина останавливается, когда она находится в состоянии $q_3$, сканируя $S_1$, потому что нет ни записи в таблице, ни стрелки, ни четверки, говорящей ей, что делать в таком случае.

# Конфигурация

Можно показать, как такая машина Тьюринга работает, записав последовательность **конфигураций** (*configuration*). Для каждого этапа существует одна конфигурация машины, которая показывает, что содержится на всей ленте, на какую ячейку указывает головка и каково значение регистра состояния. 

Записать конфигурацию можно так: $1_2 100111$. Эта запись говорит, что на ленте заштрихованы две ячейки, за которыми следуют две пустые ячейки и три заштрихованные. Головка указывает на крайнюю левую заштрихованную ячейку, и находится в состоянии $q_2$. Остальные ячейки ленты являются пустыми, так что данная запись эквивалентно также и такой записи: $0001_2 10011100$ – можно добавлять произвольное количество нулей с обоих концов.

# Реализация в коде

In [1]:
def E(tape: list, p: int):
    """erase"""
    tape[p] = 0
    return p

def W(tape: list, p: int):
    """write"""
    tape[p] = 1
    return p

def L(tape: list, p: int):
    """left"""
    return p - 1

def R(tape: list, p: int):
    """right"""
    return p + 1

def execute(tape:list, instructions:list, start_position=5):
    p = start_position
    index = 1
    steps = 0
    while True:
        # print(tape)
        # print(p, state)
        steps += 1
        value = tape[p]
        act, state = instructions[state](value)
        if act is None:
            break
        p = act(tape, p)
    print(f"{steps = }")
    return p

# Примеры программ

## Запись в трех клетках

Напишем программу, представленную выше, которая производит запись в трех последовательных клетках и завершается, указывая на крайний левый символ.

In [107]:
type_three_strokes = [
    None,
    lambda value : (L, 2) if value==1 else (W, 1),
    lambda value : (L, 3) if value==1 else (W, 2),
    lambda value : (None, None) if value==1 else (W, 3),
]

In [108]:
tape = [0] * 10
print(tape)
position = 5
steps = execute(tape, type_three_strokes, position)
tape, steps

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
steps = 6


([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], 3)

## Удваивание числа

Будем использовать **унарную систему счисления** (*unary numeral system* или *tally numeration system*), в которой, для хранения, скажем, числа 5, будет блок из пяти заштрихованных клеток: $\ldots 0111110 \ldots$. Граф перехода может быть представлен так:

![fig](https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/science/turing_machine_doubling.png)

In [70]:
doubling_number_of_strokes = [
    None,
    lambda value : (L, 2)  if value else (None, None),
    lambda value : (L, 3),
    lambda value : (L, 4)  if value else (W, 3),
    lambda value : (R, 5)  if value else (W, 4),
    lambda value : (R, 5)  if value else (R, 6),
    lambda value : (R, 6)  if value else (L, 7),
    lambda value : (E, 7)  if value else (L, 8),
    lambda value : (L, 9)  if value else (L, 11),
    lambda value : (L, 9)  if value else (L, 10),
    lambda value : (L, 10) if value else (R, 2),
    lambda value : (L, 11) if value else (R, 12),
    lambda value : (None, None),
]

In [110]:
tape = [0] * 25
number = 4
position = 15
tape[position: position+number] = [1] * number
print(tape)

steps = execute(tape, doubling_number_of_strokes, position)
print(tape)
print(steps)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
steps = 98
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6


## Произведение двух натуральных чисел

![fig](https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/science/turing_machine_multiplication.png)

In [83]:
multiplying = [
    None,
    lambda value : (E, 1)  if value else (R, 2),
    lambda value : (R, 3)  if value else (R, 15),
    lambda value : (R, 3)  if value else (R, 4),
    lambda value : (E, 5)  if value else (R, 4),
    lambda value : (R, 6), 
    lambda value : (R, 7)  if value else (W, 12),
    lambda value : (R, 7)  if value else (R, 8),
    lambda value : (R, 9)  if value else (W, 10),
    lambda value : (R, 9)  if value else (W, 10),
    lambda value : (L, 10) if value else (L, 11),
    lambda value : (L, 11) if value else (R, 4),
    lambda value : (L, 13),
    lambda value : (L, 14) if value else (L, 13),
    lambda value : (L, 14) if value else (R, 1),
    lambda value : (L, 17) if value else (W, 16),
    lambda value : (R, 15),
    lambda value : (L, 17) if value else (R, 18),
    lambda value : (None, None),
]

In [106]:
number1 = 4
number2 = 3

tape = [0] * 25
position = 5

# запись первого числа на ленту
tape[position: position+number1] = [1] * number1

# запись второго числа на ленту
p1 = position + number1 + 1
p2 = p1 + number2
tape[p1: p2] = [1] * number2

print(tape)

position = execute(tape, multiplying, position)
print(tape)
print(f"{position = }")
sum(tape)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
steps = 165
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
position = 10


12

Вычисление – это эволюция информационного состояния вычислительной системы, управляемая набором эффективно выполнимых правил (rules). При этом на любом этапе система может полчать входные данные и генерировать выходные данные. В частном случае, входные данные подаются в систему в начальном состоянии, эволюция системы завершается, и конечное состояние определяет выходные данные.

Эволюция определяет конечную или бесконечную трассу состояний (state trace), в которых пребывает система. Обычно предполагается при этом дискретность времени, но в общем случае, время может быть непрерывным. В распределенных системах трасса может бразовывать частичный порядок (история без единой глобальной линии времени). В частном случае мы имеем линейную упорядоченность состояний трассы. Часто различают внутренние состояния и входные данные, и трассу образуют не только внутренние состояния, но и связанная с ним история наблюдений(входящих данных). 

Правила являются эффективными, если:
- конечность: имеется конечный набор правил, или их конечное описание; 
- однозначность: правила не допускают двусмысленности (или явно допускают случайный выбор с заданным распределением);
- механичность шага: из текущего состояния можно за конечное число элементарных операций вычислить следующий шаг (никаких «и так далее» или «пока не найдёте подходящее…» без указанного способа искать);
- локальность исполнения: для того, чтобы сделать следующий шаг, достаточно доступных данных и самих правил (не нужны внешние оракулы или интуиция).

# Вычислимость

Функция $f: \mathbb N \to \mathbb N$ называется **вычислимой**, если может быть определен конечный список однозначно определенных инструкций, который делает принципиально возможным определение значения $f(n)$ для любого аргумента $n$ за конечное число шагов. Это понятие расширяется очевидным образом на двуместные и многоместные функции. Инструкции должны быть полностью определенными и явными. Они должны говорить нам о каждом шаге, который необходимо выполнить, без необходимости спрашивать кого-то или выяснять самостоятельно, о том, что делать: инструкции не должны требовать дополнительных источников информации, и не должны требовать изобретательности для выполнения, инструкции должны быть такими, чтобы можно было бы попробовать автоматизировать процесс применения правил, и выполнить на каком нибудь механическом устройстве. 

Определение значения $f(n)$ по заданному $n$ называется **вычислением**. В общем случае вычислением является конечная последовательность шагов, которые преобразуют вход в выход, следуя хорошо определенному набору правил, называемому алгоритмом.

>В менее строгом смысле понятие вычисления может быть применено и к бесконечным процессам. Например, генерация простых чисел или чисел Фибоначчи будет продолжаться бесконечно. В то же время, в рамках данных бесконечных вычислительных процессов, можно выделить конечные шаги: нахождение $n$-го простого числа может быть выполнено за конечное число шагов, поэтому в строгом смысле этот конкретный случай является вычислением. (Подробнее – вычисления по Чёрчу-Тьюрингу, Stream Processing, Лямбда-исчисление и рекурсия, Ко-вычисления).

>Также можно расширить понятие вычисления, допуская бесконечность в количестве инструкций, а так-же их неоднозначность и не фиксированность. Однако в контексте машин Тьюринга мы имеем дело только со строгим определением понятия вычисления.

Для $k$-местной функции $f: \mathbb N^k \to \mathbb N$ определим следующую спецификацию работы машины Тьюринга:

**(a)** Аргументы $n_1, \ldots, n_k$ будут представлены в унарной системе счисления блоками заштрихованных клеток. Каждый блок отделяется от следующего пустой клеткой. таким образом 3 и 2 будут представлены на ленте как $111011$. 

**(b)** В самом начале машина будет сканировать крайнюю левую заштрихованную клетку на ленте, и начальное состояние будет 1. Таким образом, перед вычислением, скажем $3+2$ начальная конфигурация будет $1_1 11011$. Такая конфигурация называется **стандартной начальной конфигурацией** (*standard initial configuration*).

**(c)** Если значение функции по аргументам вычислено, то в итоге машина остановится на участке ленты, содержащей блок с ответом, который представляет значение (число). При вычислении $3+2$ на ленте в итоге будет $11111$.

**(d)** В таком случае машина остановится на крайней левой заштрихованной клетке ленты. Таким образом при вычислении $3+2$ финальная конфигурация будет $1_m 1111$, где состояние $m$ соответствует инструкции, для которой не определено действие при считывании $1$. Такая конфигурация называется **стандартной финальной конфигурацией** (*standard final configuration*).
ф
**(e)** Если при вычислении значения функции не удается достичь результата, то машина либо никогда не остановится, либо остановится на нестандартной конфигурации, например, $0_m11111$ или $0111_m11$. 

С такой спецификацией любая машина Тьюринга может быть использована как для вычисления функции от одного аргумента, двух или в общем случае – $k$ аргументов.
Рассмотрим машину, определенную единственной четверкой $q_1 S_1 S_1 q_2$. Стартуя в стандартной начальной конфигурации, машина немедленно останавливается, не производя каких либо изменений на ленте. Если на ленте изначально был один блок заштрихованных клеток, то финальная конфигурация будет стандартной, и в таком случае машина вычисляет тождественную функцию (*identity function*) от одного аргумента: $\mathrm{id}(n) = n$. В этом случае машина вычисляет всюду определенную функцию (*total function*) от одного аргумента. Но если изначально на ленте есть два заштрихованных блока (два аргумента), финальная конфигурация не будет стандартной. В этом случае машина вычисляет экстремально частично определенную функцию (*partial function*) от двух аргументов, которая не определена ни для одной пары аргументов: пустую функцию (*empty function*) $e_2$ от двух аргументов. И в общем, для $k$ аргументов эта машина вычисляет пустую функцию $e_k$ от $k$ аргументов.

Рассмотрим теперь, напротив, машину, вычисляющую всюду определенную функцию для любого $k$, которая возвращает всегда одно конкретное значение, а именно, $1$.

![fig](https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/science/computing_1.png)

## Тезис Тьюринга

Числовая функция от $k$ аргументов является **вычислимой по Тьюрингу** (*Turing computable*), если существует машина Тьюринга, которая вычисляет ее в рассмотренном выше смысле. Это говорит о том, что вычисление в смысле машины Тьюринга является одним из видов вычислений, так что все вычислимые по Тьюрингу функции являются вычислимыми. Однако **тезис Тьюринга** гласит, что напротив, всякая вычислимая функция является вычислимой по Тьюрингу. Т.о. вычислимость в разработанном нами техническом смысле совпадает с вычислимостью в общем. 

Можно представить множество других видов машин Тьюринга, которые используют больше символов, чем 0 и 1, оперируют не лентой, а сеткой, способные перемещаться не только вправо-влево, но и вверх-вниз. Из тезиса Тьюринга следует, что никакие расширения возможностей в определении вида машины Тьюринга не расширяют класс вычислимых функций, так как все функции, которые вычислимы на каком либо виде машины Тьюринга, также вычислимы и на некоторой машине Тьюринга ограниченного вида, которую мы описали.

Можно привести эвристический аргумент в пользу этого. Вычисление состоит из перемещения и записи (может и стирания), в соответствии с определенными, явными правилами. Конечно, запись и стирание символов может производиться клетка за клеткой, и перемещение от одного места до другого может быть осуществлена шаг за шагом. Но главный аргумент будет в аккумулировании примеров вычислимых функций, для которых удалось показать, что они вычислимы по Тьюрингу. Выше мы увидели пару примеров вычисления числовых функций машиной Тьюринга, которые, следовательно, являются вычислимыми: сложение и умножение.

В этой серии операций (сложение, умножение) следующим является экспоненцирование. Также как умножение является повторением сложения, также и экспоненцирование является повторением умножения, а суперэкспоненцирование – это повторение экспоненцирования и тд. Если тезис Тьюринга верен, то должна быть машина Тьюринга, для каждой такой функции из этой бесконечной серии, способная ее вычислить. Проектирование умножителя уже было довольно непростой задачей, так что можно предположить, что проектирование экспонициирующей программы – тот еще вызов, и, в любом случае, прямой подход проектирования машины для каждой операции занял бы вечность, так как в одной только этой серии уже бесконечное множество операций, не говоря уже о множестве других серий. 

# Невычислимость

Счетность вычислимых по Тьюрингу функций обуславливается тем, что всякая программа может быть записана в виде последовательности четверок (*quadruples*), а программа для заданного устройства (вида) машины Тьюринга, определяет ее однозначно. Такая последовательность представляет собой конечный текст символов из конечного алфавита. Множество таких текстов счетно. 
Будем рассматривать целочисленные функции от одного аргумента $f: \mathbb N \to \mathbb N$. Множество вычислимых по Тьюрингу функций – счетно, в то время как множества целочисленных функций от одного аргумента несчетно. Поэтому таких функций слишком много, чтобы все они были бы вычислимы по Тьюрингу. Значит должны существовать функции, невычислимые по Тьюрингу.

## Счетность вычислимых функций

Рассмотрим все это детальней. Покажем, что множество вычислимых по Тьюрингу целочисленных функций от одного аргумента $f(n)$ счетно, путем перенумерования машин Тьюринга, представленных с использованием последовательностей четверок. Пусть у нас есть $n$ состояний $q_1, \ldots, q_n$. Начальным состоянием всегда будет $q_1$, а **состоянием остановки** (*halted state*) будет всегда $q_{n}$, у которого не будет инструкций. Также мы хотели бы, для стандартизации, чтобы для каждого состояния, кроме последнего, были определены инструкции для каждого символа. В случае, если для некоторого состояния $q_i$ не определена инструкция при некотором символе $S_j$, то в таком случае отсутствие инструкции будет равносильна явной инструкции перехода в состояние $q_{n}$ с сохранением символа: $q_i S_j S_j q_n$. Тогда образцом программы (граф которого представлен выше) будет:

$$
q_1 S_0 R q_3, q_1 S_1 S_0 q_2, q_2 S_0 R q_1, q_2 S_1 S_1 q_4, q_3 S_0 S_1 q_4, q_3 S_1 S_0 q_2
$$

Заметим, что для состояния остановки $q_{4}$ в последовательности нет четверок: послединие четверки описывают состояние $q_3$. Для любой программы первые два смивола четверок будут всегда предсказуемы: $q_1 S_0, q_1 S_1, q_2 S_0, q_2 S_1, \ldots, q_n S_0, q_n S_1$, поэтому их можно не писать, так что можно запистаь программу так:

$$
R q_3, S_0 q_2, R q_1, S_1 q_4, S_1 q_4, S_0 q_2
$$
Обозначим $q_i$ как $i$, $S_0$ как $1$, $S_1$ как $2$ (чтобы избежать $0$), и $L$ как $3$, а $R$ как $4$. Тогда программу можно запистаь еще проще:
$$
4, 3, 1, 2, 4, 1, 2, 4, 2, 4, 1, 2
$$
Таким образом, машина Тьюринга может быть полностью представлена как конечаня последовательность натуральных чисел, и более того, представлена единственным натуральным числом, например, используя кодирование на основе разложения на простые множители:
$$
2^4 · 3^3 · 5 · 7^2 · 11^4 · 13 · 17^2 · 19^4 · 23^2 · 29^4 · 31 · 37^2
$$

Обратное, конечно, неверно: не каждому натуральному числу соответствует некоторая машина Тьюринга, поэтому мы пронумеровали множество машин Тьюринга с существенными промежутками. По мере возрастания номера, мы можем обозначить машины Тьюринга $M_1, M_2, M_3, \ldots$, и каждая такая машина может быть запущена в стандартном начальном состоянии, при котором на ленте будет один блок заштрихованных клеток. Тогда получаем список функций от одного аргумента $f_1, f_2, f_3, \ldots$, где $f_i$ – всюду или частично определенная функция, вычисляемая машиной $M_i$, либо нигде не определенная функция.

Первый тривиальный пример можно получить рассматривая машину, представленную последовательностью $(1, 1, 1, 1)$ или $2 \cdot 3 \cdot 5 \cdot 7 = 210$. В четверках это будет представлено как $q_1 S_0 S_0 q_1, q_1 S_1 S_0 q_1$. Начиная считывать заштрихованную клетку, машина стирает ее и навсегда остается в состоянии $q_1$, никогда не переходя в состояние остановки $q_2$.

Второй пример $(2, 1, 1, 1)$ или $2^2 \cdot 3 \cdot 5 \cdot 7 = 420$, которому соответствует четверка $q_1 S_0 S_1 q_1, q_1 S_1 S_0 q_1$. Стартуя с заштрихованной клетки, машина затирает ее, затем снова заштриховывает, и так далее, и снова машина никогда не останавливается. 

Третий пример $(1, 2, 1, 1)$ или $2 \cdot 3^2 \cdot 5 \cdot 7 = 630$, которому соответствует четверка $q_1 S_0 S_0 q_2, q_1 S_1 S_0 q_1$. Стартуя с заштрихованной клетки, машина затирает ее и переходит в состояние остановки $q_2$, в котором считывается пустая клетка. Это означает, что машина завершается в нестандартной финальной конфигурации. $210$, $240$ и $630$ – наименьшие числа, которым соответствуют машины Тьюринга, так что именно эти три машины обозначаются $M_1, M_2, M_3$ и мы имеем три пустые (нигде не определенные) функции $f_1 = f_2 = f_3 = e$. 

## Функция, невычислимая по Тьюрингу

Мы явно пронумеровали множество вычислимых по Тьюрингу функций $f(n)$, пронумеровав машины, которые их вычисляют. Сам факт возможности такой нумерации говорит о том, что должны существовать невычислимые по Тьюрингу функции $f(n)$. Нумерации которые мы построили помогут продемонстрировать пример такой функции. Определим диагональную функцию (*diagonal function*):
$$
d(n) =
\begin{cases}
2 & \text{if }f_n(n) \text{ is defined and} = 1 \\
1 & \text{otherwise}
\end{cases}
$$
Очевидно, что $d$ всюду определена на $\mathbb N$, однако она не вычислима по Тьюрингу, поэтому $d$ не совпадает ни с $f_1$ ни с $f_2$ и тд. 

*Proof*: Предположим, что $d$ – одна из вычислимых по Тьюрингу функций, скажем $f_m$. Тогда, для каждого натурального числа $n$, как $d(n)$ так и $f_m(n)$ определяются одинаковым образом или ни одна из них не определена. Рассмотрим случай $n=m$:
$$
f_m(m) = d(m) =
\begin{cases}
2 & \text{if }f_m(m) \text{ is defined and} = 1 \\
1 & \text{otherwise}
\end{cases}
$$
Возникает противоречие: если $f_m(m)$ не определена, то она определена и равна $1$, если же она определена, но не равна $1$, то она равна единице, а если она определена и равна $1$, то она равна $2$. Так как предположение, что функция $d$ появляется в списке $f_1, f_2, \ldots, f_m, \ldots$, приводит к противоречию, мы можем заключить, что предположение ложное. Таким образом мы доказали теорему:

**T**. Диагональная функция $d$ невычислима по Тьюрингу.

Согласно тезису Тьюринга, так как $d$ невычислима по Тьюрингу, она является невычислимой в принципе. Почему? Несмотря на то, что не существует машины Тьюринга, которая могла бы вычислить $d$, мы по карйней мере можем вычислить несколько его первых значений. Как мы выше отмечали функции $f_1, f_2, f_3$, являются пустыми функциями, поэтому $d(1) = d(2) = d(3) = 1$. Может показаться, что мы действительно можем вычислить $d(n)$ для любого натурального числа $n$, если мы не ограничены по времени.

Конечно, мы можем теоретически рассчитать, какую последовательность четвёрок определяет машина $M_n$, выяснив, какое натуральное число ему соответствует и произведя разложение на простые множители. Однако в практическом плане это может оказаться невозможным для больших $n$, так как это потребует огромного времени вычисления. Но предположим, что мы не ограничены во времени.

Также не сложно будет следовать операциям, определенным машиной $M_n$, если определена начальная конфигурация, и если $M_n$ в итоге остановится, то мы об этом узнаем. Таким образом если мы запустим машину $M_n$ с входным значением $n$ и она остановится, то мы можем увидеть, остановилась ли она в нестандартной конфигурации, оставив $f_n(n)$ неопределенной, или остановится в стандартной конфигурации с выходным значением $f_n(n) = 1$ или же со значением $f_n(n) \neq 1$. Из этого мы сделаем вывод $d(n)=1$ или $d(n)=2$.

Но возможен и еще один случай, при котором $d(n)=1$, а именно, если $M_n$ никогда не завершится. Если при заданной начальной конфигурации $M_n$ никогда не завершится, то как мы можем это выяснить за конечное время? Это существенный вопрос: определить, остановится ли когда нибудь или нет машиа $M_n$, которая стартует считывая крайнюю левую заштрихованную клетку непрерывного блока на ленте (пустой в остальных областях). 

Является ли это обычным делом? Должна ли быть некоторая точка в процессе следования операциям машины, при которой становится очевидным, остановится ли она когда-нибудь или нет? В простых случаях это так, как мы видели в случае машин $M_1, M_2, M_3$. Однако для того, чтобы функция $d$ была вычислимой, должна быть стандартная механическая процедура, применимая не только в этих простых случаях, но так же и в более сложных случаях, позволяющая выяснить, остановится ли когда нибудь данная машина, запущенная в заданной конфигурации, или нет.

Возьмем умножение, рассмотренное выше. Представление этой машины в виде последовательности натуральных чисел состоит из 68 чисел (т.е. $17 \cdot 4$), каждая из которых меньше 18 (т.к. всего инструкций в программе 17). Предположим, кто-то наткнулся на такую последовательность и убедившись, что она действительно соответствует некоторой машине Тьюринга, построил его графовое представление. Но существует ли такая стандартная процедура, в этом или еще более сложных случаях, позволяющая установить из исследования графа, остановится ли машина когда-нибудь при заданной начальной конфигурации, или нет.

Если такая процедура есть, то тезис Тьюринга был бы ошибочным: если тезис Тьюринга верен, то такой процедуры быть не может. В настоящее время никому еще не удалось описать такую процедура. Этот факт можно рассматривать как своего рода подтверждение тезиса Тьюринга.

## Функция остановки

Поставим вопрос по-другому. Рассмотрим **функцию остановки** (*halting function*) $h(m, n)$ от двух аргументов, тесно связанную с диагональной функцией $d$. Значение $h(m, n) = 1$ или $2$ в зависимости от того, останавливается когда-нибудь машина $M_m$, стартующая с входным значением $n$, или нет. Если бы $h$ была бы вычислимой, то вычислимой была бы и $d$: для заданного $n$ мы бы сначала вычислили бы $h(n, n)$ и если бы в результате получили 2, то мы бы знали, что $d(n)=1$. А если бы получили $h(n, n)=1$, то зная, что машина $M_n$ когда либо остановится, могли бы ее запустить при стандартной начальной конфигурации с входным значением $n$ и дождаться остановки, после чего могли бы уже по результатам определить значение $d(n)$: если машина остановилась в стандартной конфигурации и $f_n(n)=1$, то $d(n)=1$, в противном случае – $d(n)=2$.

Это неформальный аргумент, показывающий, что если бы $h$ была вычислимой, то и $d$ была бы вычислима. Так как мы показали, что $d$ не является вычислимой по Тьюрингу, следуя тезису Тьюринга, мы можем заключить, что $d$ является невычислимой, и, следовательно, $h$ невычислима. 

Можно было бы доказать строго, что если бы $h$ была вычислима по Тьюрингу, то и $d$ была бы вычислима по Тьюрингу, и так как мы показали, что $d$ *не* является вычислимой по Тьюрингу, то это показывает, что и $h$ не является вычислимой по Тьюрингу. Но мы можем строго доказать, что $h$ не является вычислимой по Тьюрингу и другим путем, не вовлекая $d$.

## Строгое доказательство

*Proof*: введем в рассмотрение две специальные машины Тьюринга. 

**Копирующая машина** (*copying machine*) $C$. Если дана пустая лента, на которой записан один блок из $n$ заштрихованных клеток, и машина стартует считывая крайнюю левую заштрихованную клетку, то машина в итоге остановится и на ленте будет записано два блока, разделенных одной пустой клеткой и содержащих по $n$ заштрихованных клеток, а машина будет указывать на крайнюю     левую заштрихованную клетку.

**Сглаживающая машина** (*dithering machine*) $D$. Стартуя при тех же условиях, что и $C$, сглаживающая машина в конечном счете остановится, если $n>1$, но никогда не остановится, если $n=1$. Такая машина описывается последовательностью из двух состояний:
$$
1, 3, 4, 2, 3, 1, 3, 3
$$
Стартуя на заштрихованной клетке в состоянии 1, машина шагает вправо и переходит в состояние 2. Если после этого машина оказывается на заштрихованной клетке, она смещается обратно влево и переходит в состояние остановки 3. Если же после первой операции машина оказывается на пустой клетке, то она смещается обратно влево и переходит в исходное состояние 1: машина зацикливается, двигаясь туда-сюда.

Теперь предположим, что у нас есть машина $H$, которая вычисляет функцию $h(m, n)$. Мы могли бы комбинировать машины $C$ и $H$ следующим образом: если состояния машины $C$ пронумерованы от $1$ до $p$, а состояния машины $H$ – от $1$ до $q$, то переобозначим состояния машины $H$ от $p+1$ до $r=p+q$ и запишем их после инструкций машины $C$. Получим машину $CH$, состояния которой пронумерованы от $1$ до $r$. Изначально машина $C$ подразумевала остановку при переходе в состояние $p+1$, но в комбинированной машине переход в состояние $p+1$ начинает операции машины $H$. Таким образом новые комбинированные инструкции будут сначала проходить через операции машины $C$, копируя входное число $n$, и потом, когда машина $C$ остановится и на ленте будет записано уже два числа $n$, запустятся операции машины $H$. Результатом будет машина $CH$, которая вычисляет функцию $g(n) = h(n, n)$.

Теперь комбинируем машину $CH$ с машиной $D$, перенумеровав его два состояния в $r+1$ и $r+2$ и записав их после состояний машины $CH$. В результате получим машину $CHD$, которая проходит через операции машины $CH$ и затем через операции машины $D$. 

Таким образом, если машина Тьюринга, идущая под номером $n$ (ранее, мы обозначили ее как $M_n$), останавливается, когда стартует со значением $n$, т.е. если $h(n,n)=g(n)=1$, то машина $CHD$ не остановится, если будет стартовать со значением $n$. Если же $M_n$ не останавливается, когда стартует со значением $n$, т.е. если $h(n,n)=g(n)=2$, то машина $CHD$ останавливается (стартуя с $n$).

Покажем, что такая машина как $CHD$ не может существовать. Путь $m$ – номер машины $CHD$, т.е. имеем дело с машиной $M_m$ (ей соответствует функция $f_m$). Что произойдет, если запустить эту машину со стартовым значением $m$ (вычислить $f_m(m)$)? Машина $CHD$ остановится в том и только в том случае, если машина $M_m$ (т.е. она же сама) *не* остановится, стартуя со значением $m$. Противоречивость в существовании машины $CHD$ показывает, что не может быть такой машины как $H$.